# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [16]:
import pandas as pd
import numpy as np

from lightgbm import LGBMRegressor

from catboost import CatBoostRegressor

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
df = pd.read_csv('autos.csv')

In [19]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


**Признаки:** <br>
DateCrawled — дата скачивания анкеты из базы <br>
VehicleType — тип автомобильного кузова <br>
RegistrationYear — год регистрации автомобиля <br>
Gearbox — тип коробки передач <br>
Power — мощность (л. с.) <br>
Model — модель автомобиля <br>
Kilometer — пробег (км) <br>
RegistrationMonth — месяц регистрации автомобиля <br>
FuelType — тип топлива <br>
Brand — марка автомобиля <br>
Repaired — была машина в ремонте или нет <br>
DateCreated — дата создания анкеты <br>
NumberOfPictures — количество фотографий автомобиля <br>
PostalCode — почтовый индекс владельца анкеты (пользователя) <br>
LastSeen — дата последней активности пользователя <br>

**Целевой признак** <br>
Price — цена (евро) <br>

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [21]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


> **Удаляем машины которые стоят меньше 200 евро (возможно эти машины неисправны и выставляют их на металлалом/запчасти)**

In [22]:
df = df.loc[df['Price'] > 200]

> **Удаляем неинформативные столбцы**

In [23]:
df = df.drop(['RegistrationMonth', 'DateCrawled', 'DateCreated', 
              'FuelType', 'PostalCode', 'LastSeen', 'NumberOfPictures'], axis=1)

In [24]:
df.isnull().sum()

Price                   0
VehicleType         30870
RegistrationYear        0
Gearbox             15274
Power                   0
Model               16364
Kilometer               0
Brand                   0
Repaired            61605
dtype: int64

> **Удаляю все строки с пропусками (использовал imputer'ы, но удаление сказывается на финальной метрике лучше)**

> **Оставляем строки год регистрации которых меньше 2016 (текущий для датасета) и мощность машины от 30 до 200 л.с.**

In [25]:
df = df.loc[df['RegistrationYear'] < 2016]

In [26]:
df = df.loc[df['Power'] < 200]
df = df.loc[df['Power'] > 30]

In [27]:
df = df.dropna().reset_index()
df.shape

(215839, 10)

> **После предобработки данных у нас осталось 215 тысяч строк**

## Обучение моделей

In [28]:
y = df['Price']
X = df.drop('Price', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=1)

In [29]:
categorical = list(X.select_dtypes('object').columns) # список названий категориальных столбцов 
numerical = list(X.select_dtypes('int64').columns) # список названий численных столбцов

cat_pipe = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

num_pipe = Pipeline([
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('cat', cat_pipe, categorical),
    ('num', num_pipe, numerical),
])

### LinearRegression

In [ ]:
lin_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

In [ ]:
scores = cross_val_score(lin_pipe, X_train, y_train, scoring='neg_root_mean_squared_error', cv=3)
final_score = sum(scores)/len(scores)
print('Средняя оценка качества модели:', final_score*-1)

Средняя оценка качества модели: 2364.8478845195427


### LGBMRegressor

In [ ]:
lgbm_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LGBMRegressor(random_state=1))
])

In [ ]:
%%time
param_grid = {
    'model__max_depth':[25, 27, 33],
    'model__num_leaves':[50, 60, 70, 80],
    'model__min_child_samples': [5, 10, 15],
    'model__min_split_gain': [2, 5, 10]
}

lgbm_grid = GridSearchCV(lgbm_pipe, param_grid, cv=3, scoring='neg_root_mean_squared_error')
lgbm_grid.fit(X_train, y_train)

CPU times: user 11min 40s, sys: 5.75 s, total: 11min 46s
Wall time: 7min 34s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['VehicleType',
                                                                          'Gearbox',
                                                                          'Model',
                                                                          'Brand',
                                                                          'Repaired']),
                                                                        ('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['index',
                                                                          'RegistrationYear',
                                                                          'Power',
                                                                          'Kilometer'])])),
                                       ('model',
                                        LGBMRegressor(random_state=1))]),
             param_grid={'model__max_depth': [22, 25, 27],
                         'model__min_child_samples': [5, 10, 15],
                         'model__min_split_gain': [5, 10, 25, 50],
                         'model__num_leaves': [45, 50]},
             scoring='neg_root_mean_squared_error')

In [ ]:
print('Лучшее значение метрики', lgbm_grid.best_score_)
print('Лучшие параметры модели', lgbm_grid.best_params_)

Лучшее значение метрики -1467.8219803067752
Лучшие параметры модели {'model__max_depth': 27, 'model__min_child_samples': 10, 'model__min_split_gain': 5, 'model__num_leaves': 50}


### CatBoostRegressor

In [ ]:
%%time
cat_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', CatBoostRegressor(random_state=1, loss_function='RMSE', verbose=False))
])

param_grid = {
    'model__depth':[16],
    'model__l2_leaf_reg':[0, 1]
}

cat_grid = GridSearchCV(cat_pipe, param_grid, cv=3, scoring='neg_root_mean_squared_error', error_score='raise')
cat_grid.fit(X_train, y_train)

CPU times: user 1h 35min 55s, sys: 53min 36s, total: 2h 29min 32s
Wall time: 1h 30min 38s


GridSearchCV(cv=3, error_score='raise',
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['VehicleType',
                                                                          'Gearbox',
                                                                          'Model',
                                                                          'Brand',
                                                                          'Repaired']),
                                                                        ('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['index',
                                                                          'RegistrationYear',
                                                                          'Power',
                                                                          'Kilometer'])])),
                                       ('model',
                                        <catboost.core.CatBoostRegressor object at 0x7fcb56562940>)]),
             param_grid={'model__depth': [16]},
             scoring='neg_root_mean_squared_error')

In [ ]:
print('Лучшее значение метрики', cat_grid.best_score_)
print('Лучшие параметры модели', cat_grid.best_params_)

Лучшее значение метрики -1415.3634407621696
Лучшие параметры модели {'model__depth': 16}


### GradientBoostingRegressor

In [30]:
%%time
gradient_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', GradientBoostingRegressor(random_state=1, loss='squared_error'))
])

param_grid = {
    'model__max_depth':[8, 10, 12],
    'model__min_samples_leaf': [7, 10, 15, 20]
}

gradient_grid = GridSearchCV(gradient_pipe, param_grid, cv=3, scoring='neg_root_mean_squared_error')
gradient_grid.fit(X_train, y_train)

CPU times: user 52min 19s, sys: 2.7 s, total: 52min 22s
Wall time: 52min 25s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['VehicleType',
                                                                          'Gearbox',
                                                                          'Model',
                                                                          'Brand',
                                                                          'Repaired']),
                                                                        ('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['index',
                                                                          'RegistrationYear',
                                                                          'Power',
                                                                          'Kilometer'])])),
                                       ('model',
                                        GradientBoostingRegressor(random_state=1))]),
             param_grid={'model__max_depth': [8, 10, 12],
                         'model__min_samples_leaf': [7, 10, 15, 20]},
             scoring='neg_root_mean_squared_error')

In [31]:
print('Лучшее значение метрики', gradient_grid.best_score_)
print('Лучшие параметры модели', gradient_grid.best_params_)

Лучшее значение метрики -1433.4834213809
Лучшие параметры модели {'model__max_depth': 12, 'model__min_samples_leaf': 10}


In [32]:
models = ['LinearRegression', 'LGBMRegressor', 'CatBoostRegressor', 'GradientBoostingRegressor']
fit_time_in_min = [0, 11.5, 150, 60]
cv_rmse = [2364, 1467, 1415, 1433]

pd.DataFrame({'fit_time_in_s':fit_time_in_min,
            'cv_rmse': cv_rmse},
            index=models)

,fit_time_in_s,cv_rmse
LinearRegression,0.0,2364
LGBMRegressor,11.5,1467
CatBoostRegressor,150.0,1415
GradientBoostingRegressor,60.0,1433


**Вывод:** для задачи заказчика лучше всего подходит модель GradientBoostingRegressor, она немного уступает по точности CatBoostRegressor, но процесс обучения у нее занимает в 2.5 раза меньше времени.

# Тестирование лучшей модели

In [35]:
%%time
preds = gradient_grid.predict(X_test)
print('RMSE LGBMRegressor =', mean_squared_error(y_test, preds, squared=False))

RMSE LGBMRegressor = 1415.6918753551072
CPU times: user 991 ms, sys: 7.99 ms, total: 999 ms
Wall time: 1.68 s


> **Вывод: основываясь на качество предсказания, скорость предсказания и время обучения лучше всего подходит модель CatBoostRegressor. На валидационной выборке модель показывает значение метрики RMSE равной 1415**